## 清理 FinQA 数据集

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json

import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 读取数据集，删掉没用的列
df = pd.read_excel("/home/peterchen/Model/Data_Science_Projects/FINQA Project - A Dataset of Numerical Reasoning over Financial Data/DataFrame.xlsx", index_col=0)
df = df.drop(columns =[
    'table_retrieved', 'text_retrieved' , 'table_retrieved_all' , 
    'text_retrieved_all' , 'Rows from Text or Table' , 'Model Input' , 
    'Top-N Facts' , 'Supporting Facts' , 'table_ori'
    ])

In [3]:
dh = pd.read_excel("/home/peterchen/Model/Data_Science_Projects/FINQA Project - A Dataset of Numerical Reasoning over Financial Data/PrePost_Column_Data.xlsx", index_col=0)
dh.head(1)

,pre_text,post_text
0,['interest rate to a variable interest rate ba...,['fair value of forward exchange contracts aft...


In [4]:
def preprocess(raw_text):
    # Removing special characters and digits
    sentence = re.sub("[^a-zA-Z0-9]", " ", str(raw_text))
    
    # change sentence to lower case
    sentence = sentence.lower()

    # tokenize into words
    tokens = sentence.split()
    return pd.Series([" ".join(tokens)])

In [5]:
dh['Pre_Text'] = dh['pre_text'].apply(lambda x: preprocess(x))
dh['Post_Text'] = dh['post_text'].apply(lambda x: preprocess(x))

dh = dh.drop(columns =['pre_text', 'post_text'])

In [6]:
result = pd.concat([dh, df], axis=1)
result = result.drop(columns =['pre_text', 'post_text'])
result['Financial Context'] = result['Pre_Text'] + result['table'] + result['Post_Text']
print("result.shape: ", result.shape)
result = result.fillna(0)

result.shape:  (8281, 17)


In [7]:
print("Original rows: ", len(df))
# 找到重复行
dup_rows = result[result.duplicated()] 
# 获取重复的行号
dup_idx = result[result.duplicated()].index.tolist()
print("Duplicated numbers: ", len(dup_rows))
result = result.drop_duplicates()
print("After drop: ", len(result))

Original rows:  8281
Duplicated numbers:  63
After drop:  8218


In [8]:
result.head()

,Pre_Text,Post_Text,table,qa,Company Name,Year of Publication,Page_Number,Operations,Number of Operations,Number of Facts,Text or Table,Number of Model Inputs,Number of Top-N Facts,Question,Explanations,Answer,Financial Context
0,interest rate to a variable interest rate base...,fair value of forward exchange contracts after...,"[['', 'october 31 2009', 'november 1 2008'], [...",{'question': 'what is the the interest expense...,ADI,2009,49,"divide, divide",2,1,text,3,2,what is the the interest expense in 2009?,0,380,interest rate to a variable interest rate base...
1,abiomed inc and subsidiaries notes to consolid...,the remaining unrecognized compensation expens...,"[['', 'number of shares ( in thousands )', 'we...","{'question': 'during the 2012 year , did the e...",ABMD,2012,75,"multiply, multiply, multiply, greater",4,2,"table,text",3,1,"during the 2012 year , did the equity awards i...",0,0,abiomed inc and subsidiaries notes to consolid...
2,the following table shows annual aircraft fuel...,as of december 31 2018 we did not have any fue...,"[['year', 'gallons', 'average priceper gallon'...",{'question': 'what was the total operating exp...,AAL,2018,13,divide,1,1,table,3,2,what was the total operating expenses in 2018 ...,the total operating expenses is obtained by di...,41932,the following table shows annual aircraft fuel...
3,the fair value of our grants receivable is det...,in the third quarter of 2013 we sold our share...,"[['( in millions )', 'dec 282013', 'dec 292012...",{'question': 'what percentage of total cash an...,INTC,2013,71,divide,1,2,"table,table",3,1,what percentage of total cash and investments ...,0,53%,the fair value of our grants receivable is det...
4,entergy louisiana llc management s financial d...,the retail electric price variance is primaril...,"[['', 'amount ( in millions )'], ['2007 net re...",{'question': 'what is the growth rate in net r...,ETR,2008,313,"subtract, divide",2,2,"table,table",3,1,what is the growth rate in net revenue in 2008?,0,-3.2%,entergy louisiana llc management s financial d...


## 调用Llama 3

### 测试70B

In [15]:
import torch
import transformers
from tqdm import tqdm
import re
import gc

# 清理显存
torch.cuda.empty_cache()
gc.collect()

# 设置环境变量以避免内存碎片化
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# 设置模型和设备
model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device)

# 创建文本生成的pipeline
pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        },
    device_map = "auto", # 自动选择设备, 
)


device:  cuda:0


KeyboardInterrupt: 

In [13]:

context = result.iloc[0]['Financial Context']
question = result.iloc[0]['Question']
answer = result.iloc[0]['Answer']

def generate_prompt(context, question, prompt_type="math"):
    if prompt_type == "math":
        return f"""
        Given the following financial context (including tables) and question, write a mathematical equation to solve the financial question accurately. Don't generate useless shit before writing the equation. Just give me the math equation. The equation should be formatted as follows: 'Answer = <equation>'.

        Context: {context}
        Question: {question}
        """
    elif prompt_type == "python":
        return f"""
        Given the following financial context (including tables) and question, write a Python function that returns the answer to solve the financial question accurately. The function should take relevant variables as input and return the result. Don't generate useless shit before writing the code. Just give me the code. The function should be formatted as follows:

        def function_name(variable1, variable2, ...):
            # calculation
            return result

        Context: {context}
        Question: {question}
        """
    else:
        raise ValueError("Unknown prompt type")

# 生成数学公式的提示
math_prompt = generate_prompt(context, question, prompt_type="math")
# 生成Python代码的提示
python_prompt = generate_prompt(context, question, prompt_type="python")
# 使用 messages 参数来生成数学公式
math_messages = [
    {"role": "user", "content": math_prompt},
]

# 使用 messages 参数来生成Python代码
python_messages = [
    {"role": "user", "content": python_prompt},
]

# 生成数学公式
math_output = pipe(
    math_messages,
    max_new_tokens=256,
    do_sample=False,
)

# 生成Python代码
python_output = pipe(
    python_messages,
    max_new_tokens=256,
    do_sample=False,
)

# 提取生成的数学公式和Python代码
math_equation = math_output[0]["generated_text"][-1]['content']
python_code = python_output[0]["generated_text"][-1]['content']

print("context: ", context, "\n",
      "question: ", question, "\n",
      "answer: ", answer, "\n",
      "math_output: ", math_equation, "\n",
      "python_output: ", python_code)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


context:  interest rate to a variable interest rate based on the three month libor plus 2 05 2 05 2 34 2 34 as of october 31 2009 if libor changes by 100 basis points our annual interest expense would change by 3 8 million foreign currency exposure as more fully described in note 2i in the notes to consolidated financial statements contained in item 8 of this annual report on form 10 k we regularly hedge our non u s dollar based exposures by entering into forward foreign currency exchange contracts the terms of these contracts are for periods matching the duration of the underlying exposure and generally range from one month to twelve months currently our largest foreign currency exposure is the euro primarily because our european operations have the highest proportion of our local currency denominated expenses relative to foreign currency exposures existing at october 31 2009 and november 1 2008 a 10 10 unfavorable movement in foreign currency exchange rates over the course of the yea

In [11]:
# 打印输出结果
print("Generated math equation：")
print(math_output[0]["generated_text"][-1]['content'])

print("\nGenerated Python code：")
print(python_output[0]["generated_text"][-1]['content'])

Generated math equation：
Answer = 3.8 / (100/10000)

Generated Python code：
```python
def calculate_interest_expense(libor_rate, change_in_libor):
    """
    Calculate the interest expense based on the given LIBOR rate and change in LIBOR.

    Args:
        libor_rate (float): The LIBOR rate as of October 31, 2009.
        change_in_libor (float): The change in LIBOR in basis points.

    Returns:
        float: The interest expense.
    """
    # Given that a 100 basis points change in LIBOR would result in a $3.8 million change in annual interest expense
    interest_expense_per_basis_point = 3.8 / 100
    # Calculate the interest expense based on the given LIBOR rate and change in LIBOR
    interest_expense = interest_expense_per_basis_point * change_in_libor
    return interest_expense

# Example usage:
libor_rate = 2.05  # LIBOR rate as of October 31, 2009
change_in_libor = 0  # No change in LIBOR
interest_expense = calculate_interest_expense(libor_rate, change_in_libor)
print("